# **Covid19 Medical Thesis**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install researchpy

In [ ]:
from google.colab import data_table
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import numpy as np
import pandas as pd
import io
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu
import statsmodels.api as sm
import researchpy as rp 
%load_ext google.colab.data_table

In [ ]:
#%unload_ext google.colab.data_table

In [ ]:
#IMPORT EXCEL
path = 'drive/My Drive/Covid_Mazza/positivi-tampone.xlsx'  #remove ‘content/’ from path then use 
df = pd.read_excel(path, error_bad_lines=False)

#REPLACE DM, EMOLISI E 0 CON NaN
df.replace(0, np.nan,  inplace=True)
df.replace('DM', np.nan, inplace=True)
df.replace('EMOLISI', np.nan, inplace=True)

#RULE SEVERO
df['DISPNEA'] =df['SINTOMI RESP'].str.contains('DISPNEA', case=False)

def severo(c):
  if c['SAT O2'] <= 0.93 and c['P/F'] <=300 and c['DISPNEA'] == True:
    return 'Sì'
  else:
    return 'No'

# c['FR (ATTI/MIN)'] >= 30
  
df['SEVERO'] = df.apply(severo, axis=1)

# **Descriptive Statistics** 


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,165.0,83.000000,47.775517,1.000000,42.000000,83.000000,124.000000,165.000000
eta_calc,165.0,68.303030,14.313742,32.000000,57.000000,70.000000,79.000000,94.000000
PESO kg,106.0,78.783019,15.946939,45.000000,67.000000,78.500000,88.750000,140.000000
P/F,154.0,238.006494,91.748982,47.000000,179.250000,245.500000,295.000000,676.000000
FR (ATTI/MIN),63.0,25.492063,7.908495,12.000000,20.000000,24.000000,31.000000,40.000000
SAT O2,162.0,0.885370,0.086621,0.500000,0.860000,0.910000,0.940000,0.990000
GB TOT,165.0,8.354788,5.292998,2.650000,5.070000,6.640000,9.650000,36.720000
% NEUTROFILI,159.0,79.010063,10.727890,42.200000,73.000000,80.100000,86.500000,96.900000
N° NEUTROFILI,159.0,6.994312,5.313803,1.488630,3.757400,5.195170,8.486335,34.810560
%LINFOCITI,160.0,14.386250,8.580675,1.400000,8.000000,13.700000,19.025000,52.900000


In [ ]:
df.describe(include='all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Unnamed: 0,165,NaN,NaN,NaN,NaT,NaT,83,47.7755,1,42,83,124,165
SESSO,163,2,M,96,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DATA DI NASCITA,165,164,1940-06-02 00:00:00,2,1925-09-17,1987-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eta_calc,165,NaN,NaN,NaN,NaT,NaT,68.303,14.3137,32,57,70,79,94
DATA ACCESSO PS,165,53,2020-03-24 00:00:00,9,2020-02-29,2020-05-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Unnamed: 71,57,NaN,NaN,NaN,NaT,NaT,7.70175,0.461555,7,7,8,8,8
Unnamed: 72,164,149,dato mancante sulla terapia - è ancora ricoverata,9,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 73,1,1,TRASFERITO TI CCH,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DISPNEA,165,2,True,101,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Descriptive Statistics - SEVERO = Sì**
* 69 pazienti



In [ ]:
severo = df[df["SEVERO"] == "Sì"]
non_severo = df[df["SEVERO"] == "No"]

In [ ]:
severo.shape

(69, 76)

In [ ]:
severo.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,69.0,82.101449,42.682875,8.000000,49.000000,80.000000,115.000000,164.000000
eta_calc,69.0,68.985507,13.898257,37.000000,57.000000,71.000000,79.000000,93.000000
PESO kg,44.0,80.522727,17.226924,50.000000,67.750000,78.500000,91.000000,140.000000
P/F,69.0,202.927536,67.744832,47.000000,172.000000,224.000000,250.000000,300.000000
FR (ATTI/MIN),23.0,27.913043,7.585351,12.000000,23.000000,28.000000,33.500000,40.000000
SAT O2,69.0,0.835507,0.094954,0.500000,0.800000,0.870000,0.900000,0.930000
GB TOT,69.0,8.951014,5.476576,2.650000,5.390000,7.180000,11.400000,36.720000
% NEUTROFILI,67.0,80.922388,10.787701,42.200000,77.950000,82.900000,87.950000,96.300000
N° NEUTROFILI,67.0,7.520805,5.296238,1.898050,4.016900,6.038380,9.833330,34.810560
%LINFOCITI,68.0,13.272059,9.487664,2.000000,7.000000,11.950000,16.300000,52.900000


In [ ]:
severo.describe(include='all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Unnamed: 0,69,NaN,NaN,NaN,NaT,NaT,82.1014,42.6829,8,49,80,115,164
SESSO,69,2,M,35,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DATA DI NASCITA,69,69,1953-10-18 00:00:00,1,1927-01-29,1983-02-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eta_calc,69,NaN,NaN,NaN,NaT,NaT,68.9855,13.8983,37,57,71,79,93
DATA ACCESSO PS,69,33,2020-03-20 00:00:00,7,2020-03-10,2020-05-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Unnamed: 71,25,NaN,NaN,NaN,NaT,NaT,7.68,0.476095,7,7,8,8,8
Unnamed: 72,69,64,dato mancante sulla terapia - è ancora ricoverata,5,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 73,0,0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DISPNEA,69,1,True,69,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Descriptive Statistics - SEVERO = NO**


In [ ]:
non_severo.shape

(96, 76)

In [ ]:
non_severo.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,96.0,83.645833,51.334810,1.000000,36.750000,84.500000,128.250000,165.000000
eta_calc,96.0,67.812500,14.657718,32.000000,57.750000,68.500000,79.250000,94.000000
PESO kg,62.0,77.548387,14.993635,45.000000,65.000000,78.500000,86.500000,115.000000
P/F,85.0,266.482353,98.924407,57.000000,203.000000,277.000000,325.000000,676.000000
FR (ATTI/MIN),40.0,24.100000,7.843992,12.000000,18.000000,22.000000,28.500000,40.000000
SAT O2,93.0,0.922366,0.056730,0.740000,0.900000,0.940000,0.960000,0.990000
GB TOT,96.0,7.926250,5.143132,2.770000,4.995000,6.560000,8.560000,30.080000
% NEUTROFILI,92.0,77.617391,10.524525,42.900000,71.875000,78.850000,83.500000,96.900000
N° NEUTROFILI,92.0,6.610887,5.322522,1.488630,3.558795,5.015135,7.374087,28.816640
%LINFOCITI,92.0,15.209783,7.794744,1.400000,9.950000,14.700000,19.825000,41.000000


In [ ]:
non_severo.describe(include='all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Unnamed: 0,96,NaN,NaN,NaN,NaT,NaT,83.6458,51.3348,1,36.75,84.5,128.25,165
SESSO,94,2,M,61,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DATA DI NASCITA,96,96,1965-05-08 00:00:00,1,1925-09-17,1987-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eta_calc,96,NaN,NaN,NaN,NaT,NaT,67.8125,14.6577,32,57.75,68.5,79.25,94
DATA ACCESSO PS,96,47,2020-03-16 00:00:00,6,2020-02-29,2020-05-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Unnamed: 71,32,NaN,NaN,NaN,NaT,NaT,7.71875,0.456803,7,7,8,8,8
Unnamed: 72,95,87,dato mancante sulla terapia - è ancora ricoverata,4,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 73,1,1,TRASFERITO TI CCH,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DISPNEA,96,2,False,64,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **Dichotomous variables**

In [ ]:
lista_df =  [df, severo, non_severo]
for j in lista_df:

  j['SESSO'] = j['SESSO'].replace(['M'], 1)
  j['SESSO'] = j['SESSO'].replace(['F'], 2)

  lista_dicotomiche  = ['IRC', 'ONCOLOGICO', 'RESPIRATORIE CRONICHE', 'IPERTENSIONE', 'FUMATORE','DIABETE','CARDIOVASCOLARI']

  for i in lista_dicotomiche:
    j[i] = j[i].replace(['Sì'], 1)
    j[i] = j[i].replace(['NO'], 2)

# **T-Student Indicators**

In [ ]:
lista  = ['PT','APTT','FIBRINOGENO','D DIMERO','PLT','GB TOT', 'N° NEUTROFILI', 'N° LINFOCITI', 'PCR', 'LDH', 'CREA', 'eGFR_calcolata', 'ferritina',
          'troponina', 'PCT', 'ALT (9-59)', 'AST (11-34)', 'eta_calc','IRC', 'ONCOLOGICO', 'RESPIRATORIE CRONICHE', 'IPERTENSIONE', 'SESSO', 
           'CARDIOVASCOLARI', 'FUMATORE', 'DIABETE']

for i in lista:  
  severo_dropna = severo.dropna(subset=[i])
  non_severo_dropna = non_severo.dropna(subset=[i]) 
  severo_i = severo_dropna[i]
  non_severo_i = non_severo_dropna[i]   
  t, p = ttest_ind(severo_i, non_severo_i, equal_var=False)           
  print ('\nVariabile:', i ,"\nTtest_ind: t = %g  p = %g" % (t, p), '\n-------------------------------' )


Variabile: PT 
Ttest_ind: t = 0.224455  p = 0.822759 
-------------------------------

Variabile: APTT 
Ttest_ind: t = -0.737864  p = 0.461918 
-------------------------------

Variabile: FIBRINOGENO 
Ttest_ind: t = 1.10379  p = 0.273834 
-------------------------------

Variabile: D DIMERO 
Ttest_ind: t = 0.823155  p = 0.412612 
-------------------------------

Variabile: PLT 
Ttest_ind: t = 0.985106  p = 0.326353 
-------------------------------

Variabile: GB TOT 
Ttest_ind: t = 1.21598  p = 0.226023 
-------------------------------

Variabile: N° NEUTROFILI 
Ttest_ind: t = 1.06748  p = 0.287558 
-------------------------------

Variabile: N° LINFOCITI 
Ttest_ind: t = 0.139817  p = 0.889093 
-------------------------------

Variabile: PCR 
Ttest_ind: t = 3.18484  p = 0.0018337 
-------------------------------

Variabile: LDH 
Ttest_ind: t = 3.10169  p = 0.00239187 
-------------------------------

Variabile: CREA 
Ttest_ind: t = 0.327593  p = 0.743705 
-----------------------------

# **Mann–Whitney U test**

In [ ]:
lista  = ['PT','APTT','FIBRINOGENO','D DIMERO','PLT','GB TOT', 'N° NEUTROFILI', 'N° LINFOCITI', 'PCR', 'LDH', 'CREA', 'eGFR_calcolata', 'ferritina',
          'troponina', 'PCT', 'ALT (9-59)', 'AST (11-34)', 'eta_calc','IRC', 'ONCOLOGICO', 'RESPIRATORIE CRONICHE', 'IPERTENSIONE', 'SESSO', 
           'CARDIOVASCOLARI', 'FUMATORE', 'DIABETE']

for i in lista:    
    severo_i = severo[i]
    non_severo_i = non_severo[i]   
    u_statistic, pVal = stats.mannwhitneyu(severo_i, non_severo_i)            
    print ('\nVariabile:', i ,'\nP value:', pVal, '\nU Statistic:', u_statistic, '\n-------------------------------' )


Variabile: PT 
P value: 0.2808895976498903 
U Statistic: 3136.0 
-------------------------------

Variabile: APTT 
P value: 0.016503957119710342 
U Statistic: 2666.5 
-------------------------------

Variabile: FIBRINOGENO 
P value: 0.11457702059681135 
U Statistic: 2947.5 
-------------------------------

Variabile: D DIMERO 
P value: 0.2634778224225741 
U Statistic: 3120.0 
-------------------------------

Variabile: PLT 
P value: 0.09995802934300807 
U Statistic: 2923.5 
-------------------------------

Variabile: GB TOT 
P value: 0.06388759579442357 
U Statistic: 2850.5 
-------------------------------

Variabile: N° NEUTROFILI 
P value: 0.06327164429136804 
U Statistic: 2849.0 
-------------------------------

Variabile: N° LINFOCITI 
P value: 0.04979929945128048 
U Statistic: 2813.0 
-------------------------------

Variabile: PCR 
P value: 0.0019225614473807274 
U Statistic: 2436.5 
-------------------------------

Variabile: LDH 
P value: 0.0008351237750462663 
U Statistic: 23

# **Best-Worst Test**

In [ ]:
best = df.copy()
worst = df.copy()
best['FR (ATTI/MIN)'] = best['FR (ATTI/MIN)'].replace(np.nan, 30)
worst['FR (ATTI/MIN)'] = worst['FR (ATTI/MIN)'].replace(np.nan, 1)

In [ ]:
def severo_best_worst(c):
  if c['SAT O2'] <= 0.93 and c['P/F'] <=300 and c['FR (ATTI/MIN)'] >= 30 and c['DISPNEA'] == True:
    return 'Sì'
  else:
    return 'No'

lista_df_best_worst=[best, worst]
best.name = 'BEST'
worst.name = 'WORST'

for j in lista_df_best_worst: 
 
  j['SEVERO'] = j.apply(severo_best_worst, axis=1)

  severo_j = j[j["SEVERO"] == "Sì"]
  non_severo_j = j[j["SEVERO"] == "No"]
  
  print ('\n-----------\n', j.name, '\n-----------')
  print ("severo shape: ", severo_j.shape)
  print ("non_severo shape: ", non_severo_j.shape)
    
  lista  = ['PT','APTT','FIBRINOGENO','D DIMERO','PLT','GB TOT', 'N° NEUTROFILI', 'N° LINFOCITI', 'PCR', 'LDH', 'CREA', 'eGFR_calcolata', 'ferritina',
          'troponina', 'PCT', 'ALT (9-59)', 'AST (11-34)', 'eta_calc','IRC', 'ONCOLOGICO', 'RESPIRATORIE CRONICHE', 'IPERTENSIONE', 'SESSO', 
          'CARDIOVASCOLARI', 'FUMATORE', 'DIABETE']

  print ('\n-----------\n', j.name, '\n-----------')
  print ('\n-----------\n', 'TTEST' '\n-----------')


  for i in lista:
    severo_j_dropna = severo_j.dropna(subset=[i])
    non_severo_j_dropna = non_severo_j.dropna(subset=[i]) 
    severo_j_i = severo_j_dropna[i]
    non_severo_j_i = non_severo_j_dropna[i]   
    t, p = ttest_ind(severo_j_i, non_severo_j_i, equal_var=False)          
    print ('\nDatabase: ',j.name,'\nVariabile:', i ,"\nTtest_ind: t = %g  p = %g" % (t, p), '\n-------------------------------' )

  print ('\n---------------\n', 'MANN-WHITNEY U' '\n---------------')

  for k in lista:    
    severo_j_k = severo_j[k]
    non_severo_j_k = non_severo_j[k]   
    u_statistic, pVal = stats.mannwhitneyu(severo_j_k, non_severo_j_k)            
    print ('\nDatabase: ', j.name,'\nVariabile:', k ,'\nP value:', pVal, '\nU Statistic:', u_statistic, '\n-------------------------------' )

# **Charts** 


In [ ]:
lista_a=['CARDIOVASCOLARI','IPERTENSIONE',	'DIABETE',	'RESPIRATORIE CRONICHE',	'ONCOLOGICO',	'FUMATORE',	'TERAPIE']
for i in lista_a:
  fig = px.pie(df, names=i, title= i, height=400, width=700)
  fig.show()

In [ ]:
lista_r=['eta_calc', 'PESO kg', 'ALTEZZA Cm','CARDIOVASCOLARI','IPERTENSIONE',	'DIABETE',	'RESPIRATORIE CRONICHE',	'ONCOLOGICO',	'FUMATORE',	
         'TERAPIE','D DIMERO','N° NEUTROFILI', 'N° LINFOCITI','PT', 'APTT','FIBRINOGENO']

for i in lista_r:
  fig = px.histogram(df, x=i, nbins=50, title= 'Distribuzione ' + i)
  fig.show()

In [ ]:
lista = ['CARDIOVASCOLARI','IPERTENSIONE',	'DIABETE',	'RESPIRATORIE CRONICHE',	'ONCOLOGICO',	'FUMATORE']

for i in lista:    
  fig = px.bar(df,x="EBPM", color=i, barmode='group',
              title= i , height=400)
  fig.show()

KeyError: ignored